In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image
import ast



# IMG_SIZE=2400
# TRAIN_PATH = '/kaggle/input/tensorflow-great-barrier-reef'
# #Best_Model = '/kaggle/input/barrie-reef-yolo5/yolov5/kaggle-Reef/exp/weights/best.pt'
# Best_Model = '../input/tfreef-yolov5s6trainv1-weights/best.pt'

In [ ]:
# !pip install -qU wandb
!pip install -qU bbox-utility 

In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# CKPT_DIR  = '/kaggle/input/greatbarrierreef-yolov5-train-ds'
CKPT_PATH = '../input/yolov5-w-albumentationstrain1/cots_with_albs/weights/epoch6.pt' # by @steamedsheep
IMG_SIZE  = 10000#int(2000*3)  # 
CONF      = 0.275     # 1920*3 + conf-0.3
IOU       = 0.2
AUGMENT   = True

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))

df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [ ]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
from bbox.utils import coco2yolo, coco2voc, voc2yolo, voc2coco
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp ../input/yolo-arial/Arial.ttf /root/.config/Ultralytics/

In [ ]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

In [ ]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
img = ["../input/tensorflow-great-barrier-reef/train_images/video_0/1001.jpg"]
results = model(img, size=1920, augment=True)
results.pandas().xyxy[0]["class"].values

In [ ]:
def np2list(np):
    return [i.tolist() for i in np]

def predict(model, img, size=1920, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
        
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        clas   = preds["class"].values

        return np2list(bboxes), np2list(confs), np2list(clas)
    else:
        return [],[],[]
    
# def format_prediction(bboxes, confs):
#     annot = ''
#     if len(bboxes)>0:
#         for idx in range(len(bboxes)):
#             xmin, ymin, w, h = bboxes[idx]
#             conf             = confs[idx]
#             annot += f'{conf} {xmin} {ymin} {w} {h}'
#             annot +=' '
#         annot = annot.strip(' ')
#     return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 20)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
# %cd test
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import time
import shutil
import torch.nn as nn
from skimage import io
import torchvision
import cv2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch import ToTensorV2
from torchvision import utils
import albumentations as A
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)
from shutil import copyfile
from torchvision import transforms
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = DEVICE
BASE_DIR = "../input/tensorflow-great-barrier-reef/train_images/"

# num_epochs = 12
# width = 852
# height = 480

In [ ]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

# Turn annotations from strings into lists of dictionaries
df['annotations1'] = df['annotations'].apply(eval)

# Create the image path for the row
df['image_path'] = "video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"

length = lambda x: len(x) 

df["no_of_bbox"] = df["annotations1"].apply(length)
df['annotations'] = df['annotations'].apply(eval)

df.head(5)

In [ ]:
val_df = df[df["video_id"]==2][df["no_of_bbox"]>0]
val_df.shape

In [ ]:
class ReefDataset:
    
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def can_augment(self, boxes):
        """ Check if bounding boxes are OK to augment
        
        
        For example: image_id 1-490 has a bounding box that is partially outside of the image
        It breaks albumentation
        Here we check the margins are within the image to make sure the augmentation can be applied
        """
        
        box_outside_image = ((boxes[:, 0] < 0).any() or (boxes[:, 1] < 0).any() 
                             or (boxes[:, 2] > 1280).any() or (boxes[:, 3] > 720).any())
        return not box_outside_image
        
    def get_boxes(self, row):
        """Returns the bboxes for a given row as a 3D matrix with format [x_min, y_min, x_max, y_max]"""
        
        boxes = pd.DataFrame(row['annotations'], columns=['x', 'y', 'width', 'height']).astype(float).values
        
        # Change from [x_min, y_min, w, h] to [x_min, y_min, x_max, y_max]
        boxes[:, 2] = np.clip(boxes[:, 0] + boxes[:, 2],0,1280)
        boxes[:, 3] = np.clip(boxes[:, 1] + boxes[:, 3],0,720)
        return boxes
    
    def get_image(self, row):
        """Gets the image for a given row"""
#         print(f'{BASE_DIR}{row["image_path"]}')
        image = cv2.imread(f'{BASE_DIR}{row["image_path"]}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
#         image /= 255.0
        return image,f'{BASE_DIR}{row["image_path"]}'
    
    def __getitem__(self, i):

        row = self.df.iloc[i]
        image,pth = self.get_image(row)
        boxes = self.get_boxes(row)
        boxes1 = boxes
        n_boxes = boxes.shape[0]
        
        # Calculate the area
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'area': torch.as_tensor(area, dtype=torch.float32),
            
            'image_id': torch.tensor([i]),
            
            # There is only one class
            'labels': torch.ones((n_boxes,), dtype=torch.int64), #unchanged
            
            # Suppose all instances are not crowd
            'iscrowd': torch.zeros((n_boxes,), dtype=torch.int64) #unchanged   
        }
        
#         if self.transforms is not None:
#             image, target = self.transforms(image, target)
#         return image, target

        if self.transforms and self.can_augment(boxes):
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': target['labels']
            }
            sample = self.transforms(**sample)
            image = sample['image']
#             target['boxes'] = torch.as_tensor(sample['bboxes'])
#             if len(tensor.size()) != 0:
#                 target['boxes'] = target['boxes'].int()
#                 target['boxes'] = target['boxes'].to(torch.float32)
#                 print(f"{torch.tensor([i])} : {target['boxes'].shape}")
#                 print(f"{torch.tensor([i])} : {target['boxes']}")
#                 print("#"*80)
#                 return image, target
#             else:
#                 continue
                
#             print(*sample['bboxes'][0])
            if n_boxes > 0:
#                 print(torch.tensor(sample['bboxes']))
                target['boxes'] = torch.as_tensor(sample['bboxes']).int()
                target['boxes'] = target['boxes'].to(torch.float32)
#                 print(f"{torch.tensor([i])} : {target['boxes'].shape}")
#                 print(f"{torch.tensor([i])} : {target['boxes']}")
#                 print("#"*80)

        else:
            image = ToTensorV2(p=1.0)(image=image)['image']
            
#         if target['boxes'].shape != (0):
        return image, target, pth, boxes1

    def __len__(self):
        return len(self.df)

In [ ]:
def get_train_transform():
    return A.Compose([
#         A.Resize(height=height, width=width, p=1.0),
#         A.Flip(0.0),
        A.HorizontalFlip(0.8), # Same with transforms.RandomHorizontalFlip()

#         A.VerticalFlip(0.1), 

#         A.Cutout(num_holes=25, max_h_size=6, max_w_size=6, fill_value=0, p=0.5),

        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

ds_train = ReefDataset(val_df,get_train_transform())

In [ ]:
from PIL import Image 
from tqdm import tqdm

box_w_conf = []
gt_bbox = []

for i in tqdm(range(len(val_df))):
    
    image, targets, pths, _ = ds_train[i]
    boxes = targets['boxes'].cpu().numpy().astype(np.int32)

    img1 = image.permute(1,2,0).cpu().numpy().astype(np.uint8)
#     print(img1.shape)
#     break
#     from_arr = Image.fromarray(img1)
    
    bboxes, confis, clas = predict(model, img1, size=IMG_SIZE, augment=AUGMENT)
    if bboxes == []:
        demo = []
    else:
        demo = np.zeros((len(bboxes), 5))
        demo[:,0] = confis
        demo[:,1:5] = bboxes
    box_w_conf.append(demo)
    gt_bbox.append(boxes)

In [ ]:
# model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
# bboxes_list = []
# config_list = []

# box_w_conf = []
# for idx, path in enumerate(image_paths):
#     img = cv2.imread(path)[...,::-1]
# #     img = cv2.resize(img,(10000,10000),cv2.INTER_LINEAR)
    
# #     print(img.shape)
#     print(img.shape)
#     bboxes, confis, clas = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
#     demo = np.zeros((len(bboxes), 5))
#     demo[:,0] = confis
#     demo[:,1:5] = bboxes
#     box_w_conf.append(demo)
# # #     break
#     bboxes_list.append(bboxes)
#     config_list.append(confis)
#     display(show_img(img, bboxes, bbox_format='coco'))
#     if idx>5:
#         break

In [ ]:
# confis

In [ ]:
# demo = np.zeros((len(bboxes), 5))
# demo[:,0] = confis
# demo[:,1:5] = bboxes
# demo